In [137]:
import numpy as np
import UsefulFunctions as uf
import pygmt
import pandas as pd
import xarray as xr
from pyproj import Transformer


In [148]:
# functions needed for making the big grid and calculating distances (vectorized haversine + pythagorean theorem for 3d distance)

def trunc_float(nums, dec):
    truncated_nums = np.trunc(nums*10**dec)/10**dec
    
    return truncated_nums

def generate_grid(min_lat, max_lat, min_lon, max_lon, spacing_km):
    # convert km spacing to degrees, use average lat for longitude spacing
    lat_degree_spacing = trunc_float(spacing_km/111, 5)
    avg_lat = min_lat + (max_lat - min_lat)/2
    lon_degree_spacing = trunc_float(spacing_km/(111*np.cos(np.radians(avg_lat))), 5)
    
    lats = np.arange(min_lat, max_lat + lat_degree_spacing, lat_degree_spacing)
    lons = np.arange(min_lon, max_lon + lon_degree_spacing, lon_degree_spacing)
    lons_2d, lats_2d = np.meshgrid(lons, lats)
    grid = np.hstack((lats_2d.flatten().reshape(-1,1), lons_2d.flatten().reshape(-1,1)))

    return grid, lats, lons


def calculate_3d_distances(grid, other_points, depth):
    grid_latitudes = np.radians(grid[:, 0])
    grid_longitudes = np.radians(grid[:, 1])
    other_latitudes = np.radians(other_points[:, 0])
    other_longitudes = np.radians(other_points[:, 1])

    dlat = other_latitudes[:, None] - grid_latitudes
    dlon = other_longitudes[:, None] - grid_longitudes

    a = np.sin(dlat/2)**2 + np.cos(other_latitudes[:, None]) * np.cos(grid_latitudes) * np.sin(dlon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    distances_2d = 6371 * c  # 2D distances on the Earth's surface

    # Calculate 3D distances by adding depth component
    distances_3d = np.sqrt(distances_2d**2 + depth**2)

    return distances_3d

In [149]:
# vectorized functions needed for checking angles for a mass number of earthquake sources

def spherical_to_cartesian(lat, lon, depth):
    # Convert spherical coordinates (latitude, longitude, depth) to Cartesian coordinates (x, y, z)
    # lat_rad = np.radians(lat)
    # lon_rad = np.radians(lon)
    # x = np.cos(lat_rad) * np.cos(lon_rad) * (6371 - depth)
    # y = np.cos(lat_rad) * np.sin(lon_rad) * (6371 - depth)
    # z = np.sin(lat_rad) * (6371 - depth)
    
    # Just use a transformer from pyproj  (WGS84 > WGS Geocentric (XYZ))
    transformer = Transformer.from_crs("EPSG:4326", "EPSG:4978")
    x, y, z = transformer.transform(lat, lon, depth)  # outputs in m

    return np.array([x, y, z]).T/1000. # convert back to km, transpose for columns instead of rows


def compute_3d_angles(stations, sources):
    # Convert stations and sources to Cartesian coordinates
    stations_cartesian = np.array([spherical_to_cartesian(*station) for station in stations])
    sources_cartesian = np.array([spherical_to_cartesian(*source) for source in sources])
    print(stations_cartesian.shape)
    print(sources_cartesian.shape)

    return result

# Example usage
# Assuming stations is a NumPy array of shape (n, 3) containing (latitude, longitude, depth) coordinates of stations
# and sources is a NumPy array of shape (m, 3) containing (latitude, longitude, depth) coordinates of sources



In [250]:
def calculate_detection_time(grid, stations_coords, depth, velP):
    max_gap = 60
    min_angle = 60
    # get distances to each station for each point
    distances_3d = calculate_3d_distances(grid, stations_coords, depth) 
    
    sta_coords_cart = spherical_to_cartesian(stations_coords[:,0], stations_coords[:,1], stations_coords[:,2])
    grid_coords_cart = spherical_to_cartesian(grid[:,0], grid[:,1], grid[:,2])

    #initialize grid for stacking data
    dists_and_sta_coords = np.empty((len(stations_coords), len(grid), 4))
    # first pane is distance
    dists_and_sta_coords[:,:,0] = distances_3d
    # next 3 panes are coords
    dists_and_sta_coords[:,:,1] = np.tile(sta_coords_cart[:, 0].reshape(-1,1), len(grid))
    dists_and_sta_coords[:,:,2] = np.tile(sta_coords_cart[:, 1].reshape(-1,1), len(grid))
    dists_and_sta_coords[:,:,3] = np.tile(sta_coords_cart[:, 2].reshape(-1,1), len(grid))

    # sort columns of first panel (distances) and get the indices to use to reconstruct sorted array
    ai = np.argsort(dists_and_sta_coords[:, :, 0], axis=0)
    # initialize the new sorted array
    dsc_sorted = np.empty((len(stations_coords), len(grid), 4))
    # first pane is distance, next 3 are coords, same as before but sorted using the indices from argsort
    dsc_sorted[:,:,0] = np.take_along_axis(dists_and_sta_coords[:,:,0], ai, axis=0)
    dsc_sorted[:,:,1] = np.take_along_axis(dists_and_sta_coords[:,:,1], ai, axis=0)
    dsc_sorted[:,:,2] = np.take_along_axis(dists_and_sta_coords[:,:,2], ai, axis=0)
    dsc_sorted[:,:,3] = np.take_along_axis(dists_and_sta_coords[:,:,3], ai, axis=0)
    
    # tile the grid coords so each point can be subtracted from each station coord (like a 3d meshgrid)
    grid_tiled = np.empty((len(stations_coords), len(grid), 3))
    grid_tiled[:, :, 0] = np.tile(grid_coords_cart[:, 0].reshape(-1, 1), len(stations_coords)).T
    grid_tiled[:, :, 1] = np.tile(grid_coords_cart[:, 1].reshape(-1, 1), len(stations_coords)).T
    grid_tiled[:, :, 2] = np.tile(grid_coords_cart[:, 2].reshape(-1, 1), len(stations_coords)).T    
    
    # subtract each grid point coords from each station coords to get all displacement vectors
    displacement_vectors_array = dsc_sorted[:, :, 1:] - grid_tiled
    
    n = 4
    # get dot products for first n stations
    angles = np.ones(shape=(1,len(grid))) * np.radians(180)
    dva_first_n = displacement_vectors_array[:n, :, :]
    for i in range(n):
        mag_i_arr = np.linalg.norm(dva_first_n[i,:,:], axis=1)
        for j in range(i+1, n):
            dot_prod_arr = np.sum(dva_first_n[i,:,:] * dva_first_n[j,:,:], axis=1)
            mag_j_arr = np.linalg.norm(dva_first_n[j,:,:], axis=1)
            mag_ij_arr = mag_i_arr * mag_j_arr
            cos_angles = dot_prod_arr/mag_ij_arr
            angles = np.vstack((angles, np.arccos(cos_angles).reshape(1,-1)))
            
    # check which columns have an angle that meets the min_angle, then fill in other array
    
    
    # detection_times_all = distances_3d/velP
    # detection_times = np.sort(detection_times_all.T)[:, 3]
    
    return angles, dsc_sorted  # in radians


In [151]:
depth = 8
velP = uf.Earthquake.vel_p

# stations = pd.read_csv('Data/EEWNetwork.csv')
stations = uf.ActiveBBs
stations_coords = np.vstack((stations.lat, stations.lon, np.zeros(shape=stations.lat.shape))).T

# Define the bounding box and spacing
min_lat = 49.5
max_lat = 72
min_lon = -196
max_lon = -129
spacing_km = 10  # Adjust as needed

# Generate the grid
grid, lats, lons = generate_grid(min_lat, max_lat, min_lon, max_lon, spacing_km)
grid = np.hstack((grid, np.ones((len(grid), 1)) * depth))


In [152]:
distances_3d = calculate_3d_distances(grid, stations_coords, depth) 


In [153]:
dists_and_sta_coords = np.empty((len(stations_coords), len(grid), 4))
# first pane is distance
dists_and_sta_coords[:,:,0] = distances_3d
# next 3 panes are coords
dists_and_sta_coords[:,:,1] = np.tile(stations_coords[:, 0].reshape(-1,1), len(grid))
dists_and_sta_coords[:,:,2] = np.tile(stations_coords[:, 1].reshape(-1,1), len(grid))
dists_and_sta_coords[:,:,3] = np.tile(stations_coords[:, 2].reshape(-1,1), len(grid))


In [154]:
# sort columns of first panel (distances) and get the indices to use to reconstruct sorted array
ai = np.argsort(dists_and_sta_coords[:, :, 0], axis=0)
# initialize the new sorted array
dsc_sorted = np.empty((len(stations_coords), len(grid), 4))
# first pane is distance, next 3 are coords, same as before but sorted using the indices from argsort
dsc_sorted[:,:,0] = np.take_along_axis(dists_and_sta_coords[:,:,0], ai, axis=0)
dsc_sorted[:,:,1] = np.take_along_axis(dists_and_sta_coords[:,:,1], ai, axis=0)
dsc_sorted[:,:,2] = np.take_along_axis(dists_and_sta_coords[:,:,2], ai, axis=0)
dsc_sorted[:,:,3] = np.take_along_axis(dists_and_sta_coords[:,:,3], ai, axis=0)

In [ ]:
grid_tiled = np.empty((len(stations_coords), len(grid), 3))
grid_tiled[:, :, 0] = np.tile(grid[:, 0].reshape(-1, 1), len(stations_coords)).T
grid_tiled[:, :, 1] = np.tile(grid[:, 1].reshape(-1, 1), len(stations_coords)).T
grid_tiled[:, :, 2] = np.tile(grid[:, 2].reshape(-1, 1), len(stations_coords)).T
grid_tiled[:, 0, 1]

In [156]:
displacement_vectors_array = dsc_sorted[:, :, 1:] - grid_tiled


In [220]:
print(displacement_vectors_array[0,0,1])
sta_coords_cart_xyz = spherical_to_cartesian(stations_coords[:,0], stations_coords[:,1], stations_coords[:,2])
dva_first4 = displacement_vectors_array[:4, :, :]
print(dva_first4[0, :, :].shape)
print(dva_first4[0,:,:]*dva_first4[1,:,:])
print(np.sum(dva_first4[0,:,:]*dva_first4[1,:,:], axis=1).shape)

aa = np.arange(9).reshape(3,3)
bb = np.arange(9,18).reshape(3,3)
print(aa)
print(bb)
print(np.sum(aa*bb, axis=1))
print(np.linalg.norm(aa, axis=1))
print((9+16+25)**.5)
print('blasdkhjasdkshad')
print(np.array([1,2,3])/np.array([1,4,9]))

354.6499
(91615, 3)
[[1.13830776e+01 1.31257027e+05 6.40000000e+01]
 [1.13830776e+01 1.31123439e+05 6.40000000e+01]
 [1.13830776e+01 1.30989918e+05 6.40000000e+01]
 ...
 [8.90539400e+00 6.16773159e+01 6.40000000e+01]
 [8.90539400e+00 6.51628400e+01 6.40000000e+01]
 [8.90539400e+00 6.87163486e+01 6.40000000e+01]]
(91615,)
[[0 1 2]
 [3 4 5]
 [6 7 8]]
[[ 9 10 11]
 [12 13 14]
 [15 16 17]]
[ 32 158 338]
[ 2.23606798  7.07106781 12.20655562]
7.0710678118654755
blasdkhjasdkshad
[1.         0.5        0.33333333]


In [144]:
# wgs84 = pyproj.Proj('+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs')##4326
# geocentric= pyproj.Proj('+proj=geocent +datum=WGS84 +units=m +no_defs') ##4978
transformer = Transformer.from_crs("EPSG:4326", "EPSG:4978")


x = stations_coords[0:3,1]
y = stations_coords[0:3,0]
z = stations_coords[0:3,2]
x2 , y2  , z2 = pyproj.transform( wgs84, geocentric, x, y, z)
print(x2, y2, z2)
x2 , y2  , z2 = transformer.transform(y, x, z)
print(x2, y2, z2)


[-1880495.1461153  -1886909.70331148  1256283.72708559] [-813081.08474678 -880915.0818855  4528171.34581636] [6019886.70006054 6008401.63015676 4298086.16208707]
[-1880495.1461153  -1886909.70331148  1256283.72708559] [-813081.08474678 -880915.0818855  4528171.34581636] [6019886.70006054 6008401.63015676 4298086.16208707]


/var/folders/g1/4874q8ms0w3fy6r8bcytvqgc0000gn/T/ipykernel_30955/14160707.py:9: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2 , y2  , z2 = pyproj.transform( wgs84, geocentric, x, y, z)


In [ ]:
test_srcs = np.array([[56, -149], [49.5, -195.81563]])
test_stas = np.array([[57, -148], [30, 122], [57, -148+360], [48.5, -196]])
test_3ds = calculate_3d_distances(test_srcs, test_stas, 8)
print(test_3ds)
# print(stations_coords[77])
# print(stations.loc[77])
test_3darr = np.empty((4, 2, 3))
test_3darr[:,:,0] = test_3ds
test_3darr[:,:,1] = np.tile(test_stas[:,0].reshape(-1,1),2)
test_3darr[:,:,2] = np.tile(test_stas[:,1].reshape(-1,1),2)
print(test_3darr[:,:,2])
print(np.argsort(test_3darr[:, :, 0], axis=0))
ai = np.argsort(test_3darr[:, :, 0], axis=0)
print(np.take_along_axis(test_3darr[:,:,2], ai, axis=0))
print(np.take_along_axis(test_3darr[:,:,1], ai, axis=0))

In [251]:
angles, dsc_sorted = calculate_detection_time(grid, stations_coords, depth, velP)

In [255]:
print(dsc_sorted[:4,0,:])
print(np.degrees(angles[:, 0]))

[[  540.20976857 -3580.76554373  1399.69038536  5072.10357367]
 [  790.53667878 -3850.08091917   397.65662681  5052.45844256]
 [ 1058.65765834 -3939.68738302    98.55437029  4998.14163004]
 [ 1101.50464897 -3988.90508094    48.61076009  4959.95017796]]
[180.         100.55851549 110.93034124 114.46518066  10.54191375
  14.10613397   3.56769812]


In [ ]:
dt_2d = detection_times.reshape(len(lats), len(lons))

In [ ]:
gridBoundaries = f'{min_lon-0.5}/{max_lon+0.5}/{min_lat-0.5}/{max_lat+0.5}'
title = f"Detection Times at {depth} km Depth"
coast_border = "a/0.5p,brown"
shorelines = "0.3p,black"
fig = pygmt.Figure()
# fig.basemap(region='170/49/-128/73+r', projection='M15c', frame=["af", f'WSne+t"{title}"'])
fig.basemap(region=gridBoundaries, projection='M15c', frame=["af", f'WSne+t"{title}"'])

fig.coast(shorelines=shorelines, borders=coast_border, water='skyblue', land='lightgray')  # draw coast over datawater='skyblue'

vmin = 0
vmax = 60
pygmt.makecpt(
    cmap=['seis'],
    reverse=True,
    series=[vmin, vmax],
    no_bg=True,
    
    )

data_array = xr.DataArray(data=dt_2d, 
                          dims=['lat', 'lon'],
                          coords=dict(lon=(['lon'], lons),
                                      lat=(['lat'], lats)))

masked_data_array = xr.where((data_array < vmin) | (data_array > vmax), np.nan, data_array)

fig.grdimage(grid=masked_data_array,
             projection='M15c',
             region=gridBoundaries,
             cmap=True, 
             transparency=40)

# lons_2d, lats_2d = np.meshgrid(lons, lats)
# fig.plot(
#     x=lons_2d.flatten(),
#     y=lats_2d.flatten(),
#     style='c+0.05c',
#     cmap=True,
#     color=detection_times
# )

fig.plot(  # Plot seismic stations as triangles
    x=stations['lon'],
    y=stations['lat'],
    style='t+0.2c',
    color='white',
    pen='black',
)

fig.colorbar(
    position='g170/63+w4c/0.5c+v',
    frame=['x','ya5f2.5+lS']
    
)

fig.show()
fig.savefig(f'Figures/misc/detection time map {depth} km depth.pdf')

In [ ]:
gridBoundaries = f'{min_lon-0.5}/{max_lon+0.5}/{min_lat-0.5}/{max_lat+0.5}'
title = f"Detection Times at {depth} km Depth"
coast_border = "a/0.5p,brown"
shorelines = "0.3p,black"
fig = pygmt.Figure()
# fig.basemap(region='170/49/-128/73+r', projection='M15c', frame=["af", f'WSne+t"{title}"'])
fig.basemap(region=gridBoundaries, projection='M15c', frame=["af", f'WSne+t"{title}"'])

fig.coast(shorelines=shorelines, borders=coast_border, water='skyblue', land='lightgray')  # draw coast over datawater='skyblue'

pygmt.makecpt(
    transparency=75,
    cmap=['seis'],
    reverse=True,
    series=[0, 60]  # np.max(p[2, :])
)
data_array = xr.DataArray(data=dt_2d, 
                          dims=['lat', 'lon'],
                          coords=dict(lon=(['lon'], lons),
                                      lat=(['lat'], lats)))
fig.grdimage(grid=data_array,
             projection='M15c',
             region=gridBoundaries,
             cmap=True)

fig.plot(  # Plot seismic stations as triangles
    x=stations['lon'],
    y=stations['lat'],
    style='t+0.3c',
    color='white',
    pen='black',
)

fig.show()
fig.savefig(f'Figures/misc/detection time map {depth} km depth.pdf')

In [ ]:
# sta_lats = np.array(stations['lat'])
# sta_lons = np.array(stations['lon'])
# print(np.array(stations['lon']).shape)
# num_p = len(grid_lons)
# num_sta = len(sta_lons)
# grid_lons_2D = np.reshape(grid_lons, (num_p, 1))
# grid_lats_2D = np.reshape(grid_lats, (num_p, 1))

# sta_lons_2D = np.zeros(shape=(num_p, num_sta))
# sta_lats_2D = np.zeros(shape=(num_p, num_sta))
# for i in range(num_p):
#     sta_lons_2D[i,:] = sta_lons
#     sta_lats_2D[i,:] = sta_lats
# print(grid_lons_2D.shape)
# print(sta_lons_2D.shape)